In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("audit_10%.csv")

In [3]:
df

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,4.18,0.6,2.508,2.50,NaN,0.500,6.68,NaN,0.2,...,0.4,NaN,0.2,0.0,2.4,8.574,0.4,0.5,1.7148,1.0
1,3.89,0.00,NaN,0.000,4.83,NaN,0.966,4.83,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0.0
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0.0
3,3.89,0.00,0.2,0.000,10.80,0.6,6.480,10.80,6.0,0.6,...,0.4,0.0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060,1.0
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,5.0,0.2,...,0.4,0.0,NaN,0.0,NaN,1.416,NaN,NaN,0.2832,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,NaN,NaN,0.2,NaN,0.40,0.2,0.080,0.89,5.0,0.2,...,0.4,0.0,0.2,NaN,2.0,1.578,0.4,0.5,0.3156,0.0
772,55.57,0.47,0.2,0.094,0.37,NaN,0.074,NaN,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,NaN,0.0
773,55.57,0.24,0.2,NaN,NaN,0.2,0.008,0.28,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912,0.0
774,55.57,0.20,NaN,0.040,0.00,0.2,0.000,0.20,5.0,NaN,...,0.4,NaN,0.2,0.0,2.0,NaN,0.4,0.5,0.2880,0.0


In [4]:
df.dtypes

Sector_score      float64
PARA_A            float64
Score_A           float64
Risk_A            float64
PARA_B            float64
Score_B           float64
Risk_B            float64
TOTAL             float64
numbers           float64
Score_B.1         float64
Risk_C            float64
Money_Value       float64
Score_MV          float64
Risk_D            float64
District_Loss     float64
PROB              float64
RiSk_E            float64
History           float64
Prob              float64
Risk_F            float64
Score             float64
Inherent_Risk     float64
CONTROL_RISK      float64
Detection_Risk    float64
Audit_Risk        float64
Risk              float64
dtype: object

In [5]:
df.isnull().sum()

Sector_score      74
PARA_A            62
Score_A           76
Risk_A            87
PARA_B            77
Score_B           83
Risk_B            93
TOTAL             94
numbers           88
Score_B.1         94
Risk_C            91
Money_Value       71
Score_MV          84
Risk_D            71
District_Loss     80
PROB              63
RiSk_E            93
History           73
Prob              89
Risk_F            80
Score             75
Inherent_Risk     71
CONTROL_RISK      92
Detection_Risk    75
Audit_Risk        81
Risk               0
dtype: int64

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 20176
nilai hilang : 2017
persentase : 9.997026169706581


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Sector_score       9.536082
PARA_A             7.989691
Score_A            9.793814
Risk_A            11.211340
PARA_B             9.922680
Score_B           10.695876
Risk_B            11.984536
TOTAL             12.113402
numbers           11.340206
Score_B.1         12.113402
Risk_C            11.726804
Money_Value        9.149485
Score_MV          10.824742
Risk_D             9.149485
District_Loss     10.309278
PROB               8.118557
RiSk_E            11.984536
History            9.407216
Prob              11.469072
Risk_F            10.309278
Score              9.664948
Inherent_Risk      9.149485
CONTROL_RISK      11.855670
Detection_Risk     9.664948
Audit_Risk        10.438144
Risk               0.000000
dtype: float64

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter
def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count == 1 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None
    
def Final(df,col,k):
    terdekat = []
    mean = 0
    std = 0
    diffsquared = 0
    sum_diffsquared = 0
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        mean += y  
        euc_val.append(y)
    mean /= len(terdekat)
    
    print(euc_val)
    
    for x,y in terdekat:
        diffsquared = (y-mean)**2
        sum_diffsquared = diffsquared + sum_diffsquared
    std_dev = ((sum_diffsquared)/len(terdekat))**(1/2)
    print("mean : ",mean)
    print("standar deviasi : ",std_dev)
    
    euc_val.sort()
    fuzzy = hitung(euc_val,7)
    
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,gaussian(y,mean,std_dev)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def gaussian(val, mean, std_dev):
    gaussian_mf = 2.718**((-1 * (val-mean)**2) / (2 * std_dev**2))
    return gaussian_mf

In [11]:
data = df.iloc[:, :-1]
df2 = data.copy()
dummy = df2.fillna(0)
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk
0,3.89,4.18,0.6,2.508,2.50,0.0,0.500,6.68,0.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.4,8.574,0.4,0.5,1.7148
1,3.89,0.00,0.0,0.000,4.83,0.0,0.966,4.83,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096
3,3.89,0.00,0.2,0.000,10.80,0.6,6.480,10.80,6.0,0.6,...,0.0,0.4,0.0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,5.0,0.2,...,0.2,0.4,0.0,0.0,0.0,0.0,1.416,0.0,0.0,0.2832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,0.00,0.00,0.2,0.000,0.40,0.2,0.080,0.89,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.578,0.4,0.5,0.3156
772,55.57,0.47,0.2,0.094,0.37,0.0,0.074,0.00,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,0.0000
773,55.57,0.24,0.2,0.000,0.00,0.2,0.008,0.28,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912
774,55.57,0.20,0.0,0.040,0.00,0.2,0.000,0.20,5.0,0.0,...,0.2,0.4,0.0,0.2,0.0,2.0,0.000,0.4,0.5,0.2880


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 4780


In [13]:
euc_dist = {}

In [14]:
for i in range(dummy.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,7)

Kolom : 1
[2.72, 3.89, 3.89, 3.89, 3.41, 3.89, 3.89]
mean :  3.6542857142857144
standar deviasi :  0.41582472759761724
fuzzy :  [(2.72, 0.08015032762088173), (3.89, 0.8515915158260912), (3.89, 0.8515915158260912), (3.89, 0.8515915158260912), (3.41, 0.8415210510924385), (3.89, 0.8515915158260912), (3.89, 0.8515915158260912)]
fuzzy :  None
pembilang :  19.651050658171492
penyebut :  5.179628957843776
rata-rata tertimbang :  3.7939108801245127
 
[55.57, 55.57, 59.85, 3.89, 2.72, 55.57, 1.99]
mean :  33.59428571428571
standar deviasi :  26.652674943289522
fuzzy :  [(55.57, 0.711853646934483), (55.57, 0.711853646934483), (59.85, 0.6155945288629984), (3.89, 0.5374144509475892), (2.72, 0.5112645062882064), (55.57, 0.711853646934483), (1.99, 0.4951138079989357)]
fuzzy :  None
pembilang :  159.98291218210605
penyebut :  4.294948234901178
rata-rata tertimbang :  37.249089728735015
 
[2.37, 2.37, 55.57, 1.99, 55.57, 2.37, 1.85]
mean :  17.44142857142857
standar deviasi :  24.115374721517053
fuzzy

[2.852722877053393, 2.9952391306702357, 3.89, 2.72, 3.89, 3.89, 2.72]
mean :  3.279708858246233
standar deviasi :  0.5354776104630236
fuzzy :  [(2.852722877053393, 0.7276867143273006), (2.9952391306702357, 0.8684060110047658), (3.89, 0.5223558274937627), (2.72, 0.5791354619181319), (3.89, 0.5223558274937627), (3.89, 0.5223558274937627), (2.72, 0.5791354619181319)]
fuzzy :  None
pembilang :  13.923361622446878
penyebut :  4.321431131649619
rata-rata tertimbang :  3.221933012069526
 
[2.72, 2.72, 2.72, 0.0, 2.72, 2.72, 3.41]
mean :  2.43
standar deviasi :  1.020210062403117
fuzzy :  [(2.72, 0.9604087381813957), (2.72, 0.9604087381813957), (2.72, 0.9604087381813957), (0.0, 0.058639886289520035), (2.72, 0.9604087381813957), (2.72, 0.9604087381813957), (3.41, 0.6304538238934191)]
fuzzy :  None
pembilang :  15.211406378743542
penyebut :  5.491137401089918
rata-rata tertimbang :  2.770174058242338
 
[2.770174058242338, 2.72, 2.72, 2.72, 2.72, 3.41, 2.72]
mean :  2.8257391511774768
standar dev

[0.0, 1.85, 55.57, 3.89, 20.94081567617117, 21.61, 55.57]
mean :  22.775830810881594
standar deviasi :  22.261257111538143
fuzzy :  [(0.0, 0.5925452112383734), (1.85, 0.6429008852871302), (55.57, 0.33791096315106073), (3.89, 0.6977946972554311), (20.94081567617117, 0.996608690347241), (21.61, 0.9986297527368472), (55.57, 0.33791096315106073)]
fuzzy :  None
pembilang :  83.9094002971889
penyebut :  4.604301163167144
rata-rata tertimbang :  18.224133766148007
 
[0.0, 2.37, 2.72, 3.89, 1.85, 1.99, 1.99]
mean :  2.1157142857142857
standar deviasi :  1.0794424108618983
fuzzy :  [(0.0, 0.1465169960173265), (2.37, 0.9726372762401274), (2.72, 0.8549784993917081), (3.89, 0.25904739121585135), (1.85, 0.970160318286102), (1.99, 0.9932419152533051), (1.99, 0.9932419152533051)]
fuzzy :  None
pembilang :  11.386285626401653
penyebut :  5.189824311657726
rata-rata tertimbang :  2.193963599273491
 
[0.0, 1.85, 2.72, 55.57, 55.57, 2.37, 1.85]
mean :  17.132857142857144
standar deviasi :  24.32273052944

[0.0, 55.57, 21.61, 55.57, 3.89, 55.57, 55.57]
mean :  35.39714285714285
standar deviasi :  24.0936261365566
fuzzy :  [(0.0, 0.33990521540184543), (55.57, 0.7043543404644764), (21.61, 0.8489901189913864), (55.57, 0.7043543404644764), (3.89, 0.42530615880948314), (55.57, 0.7043543404644764), (55.57, 0.7043543404644764)]
fuzzy :  None
pembilang :  176.56500022761657
penyebut :  4.431618855060621
rata-rata tertimbang :  39.842099693657275
 
[0.0, 59.85, 59.85, 59.85, 59.85, 3.89, 21.61]
mean :  37.84285714285714
standar deviasi :  26.146951045573015
fuzzy :  [(0.0, 0.3509011209938171), (59.85, 0.7017565636625893), (59.85, 0.7017565636625893), (59.85, 0.7017565636625893), (59.85, 0.7017565636625893), (3.89, 0.43041069671692694), (21.61, 0.8247326186568794)]
fuzzy :  None
pembilang :  187.4972908402279
penyebut :  4.413070691017981
rata-rata tertimbang :  42.48680883853622
 
[0.0, 59.85, 59.85, 1.99, 3.89, 55.57, 1.85]
mean :  26.142857142857142
standar deviasi :  28.0062282431634
fuzzy :  

[0.0, 0.43, 0.77, 0.0, 1.12, 0.0, 0.0]
mean :  0.33142857142857146
standar deviasi :  0.4248168873233526
fuzzy :  [(0.0, 0.7376398696386788), (0.43, 0.9734422224326499), (0.77, 0.5869319016775286), (0.0, 0.7376398696386788), (1.12, 0.17858767853190757), (0.0, 0.7376398696386788), (0.0, 0.7376398696386788)]
fuzzy :  None
pembilang :  1.070535919893473
penyebut :  4.689521281196801
rata-rata tertimbang :  0.22828255928508426
 
[0.0, 0.0, 0.0, 0.0, 0.0, 1.01, 0.53]
mean :  0.22
standar deviasi :  0.3707521312914369
fuzzy :  [(0.0, 0.838587273603815), (0.0, 0.838587273603815), (0.0, 0.838587273603815), (0.0, 0.838587273603815), (0.0, 0.838587273603815), (1.01, 0.10331974814792182), (0.53, 0.7050213054887577)]
fuzzy :  None
pembilang :  0.47801423753844263
penyebut :  5.001277421655756
rata-rata tertimbang :  0.09557842871675536
 
[0.0, 1.6, 0.0, 0.32, 0.84, 0.0, 0.0]
mean :  0.3942857142857143
standar deviasi :  0.5706852307998352
fuzzy :  [(0.0, 0.7876938364616856), (1.6, 0.10735422380799

...

[0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
mean :  0.42857142857142855
standar deviasi :  0.1749635530559413
fuzzy :  [(0.0, 0.049802557184333074), (0.5, 0.9200523641748718), (0.5, 0.9200523641748718), (0.5, 0.9200523641748718), (0.5, 0.9200523641748718), (0.5, 0.9200523641748718), (0.5, 0.9200523641748718)]
fuzzy :  None
pembilang :  2.7601570925246155
penyebut :  5.570116742233564
rata-rata tertimbang :  0.49552948712845446
 
[0.0, 0.5, 0.5, 0.43860020803742633, 0.5, 0.5, 0.5]
mean :  0.4198000297196324
standar deviasi :  0.17268703641569302
fuzzy :  [(0.0, 0.05210242531034927), (0.5, 0.8977770164105813), (0.5, 0.8977770164105813), (0.43860020803742633, 0.9940919610063588), (0.5, 0.8977770164105813), (0.5, 0.8977770164105813), (0.5, 0.8977770164105813)]
fuzzy :  None
pembilang :  2.680451481932175
penyebut :  5.535079468369614
rata-rata tertimbang :  0.4842661243166787
 
[0.0, 0.5, 0.49552948712845446, 0.5, 0.5, 0.5, 0.5]
mean :  0.4279327838754935
standar deviasi :  0.17470963666519057
fuz

[0.3696, 0.3172, 0.372, 0.364, 0.3656, 0.3236, 0.322]
mean :  0.34771428571428575
standar deviasi :  0.02338420887029954
fuzzy :  [(0.3696, 0.6453733242102427), (0.3172, 0.426856814764743), (0.372, 0.5831918364460018), (0.364, 0.7846718577624742), (0.3656, 0.7464121772100201), (0.3236, 0.587632704971288), (0.322, 0.5463230185235747)]
fuzzy :  None
pembilang :  1.5154591289362187
penyebut :  4.320461733888345
rata-rata tertimbang :  0.350763233718617
 
[0.7508, 0.7288, 0.7508, 0.7012, 0.684, 0.6864, 0.7528]
mean :  0.7221142857142857
standar deviasi :  0.028779130760795024
fuzzy :  [(0.7508, 0.608530781274265), (0.7288, 0.9733792994485224), (0.7508, 0.608530781274265), (0.7012, 0.767949368481421), (0.684, 0.41607482680785696), (0.6864, 0.4630435735369466), (0.7528, 0.5664398756852785)]
fuzzy :  None
pembilang :  3.1904989806069044
penyebut :  4.403948506508556
rata-rata tertimbang :  0.7244632801431931
 
[0.4004, 0.4156, 0.3892, 0.4292, 0.3932, 0.3884, 0.408]
mean :  0.4034285714285714


[0.0, 0.294, 0.2904, 0.28, 0.0, 0.298, 0.0]
mean :  0.16605714285714288
standar deviasi :  0.14389844491508444
fuzzy :  [(0.0, 0.5138747967160178), (0.294, 0.673528200683329), (0.2904, 0.6884607555577603), (0.28, 0.730911054974351), (0.0, 0.5138747967160178), (0.298, 0.6568337478007356), (0.0, 0.5138747967160178)]
fuzzy :  None
pembilang :  0.7983378466523098
penyebut :  4.29135814916423
rata-rata tertimbang :  0.18603384264438316
 
[0.0, 11.0284, 45.2368, 149.8176, 12.7968, 34.9296, 134.7624]
mean :  55.51022857142857
standar deviasi :  56.80587006683915
fuzzy :  [(0.0, 0.6203928510204677), (11.0284, 0.7359804277803487), (45.2368, 0.9837810247501778), (149.8176, 0.252097728326722), (12.7968, 0.7537745307282973), (34.9296, 0.9364838904609255), (134.7624, 0.37790681427988454)]
fuzzy :  None
pembilang :  183.6730075172925
penyebut :  4.660417267346824
rata-rata tertimbang :  39.41127950155793
 
[0.0, 0.7816, 0.72, 0.736, 0.7992, 0.4756, 0.7648]
mean :  0.6110285714285714
standar deviasi 

[0.0, 2.2216, 2.4236, 1.4988, 0.3029552817629003, 0.3756, 0.3476]
mean :  1.0243078973947
standar deviasi :  0.9308200149861702
fuzzy :  [(0.0, 0.545846778754988), (2.2216, 0.43728588059246215), (2.4236, 0.3230905217500559), (1.4988, 0.8781721145207431), (0.3029552817629003, 0.7406300940701483), (0.3756, 0.7844094228559925), (0.3476, 0.7677904275556865)]
fuzzy :  None
pembilang :  3.8566067967555115
penyebut :  4.4772252401000765
rata-rata tertimbang :  0.8613832429545821
 
[0.0, 0.2824, 0.28, 0.0, 0.28, 0.0, 0.286]
mean :  0.1612
standar deviasi :  0.13961563563686452
fuzzy :  [(0.0, 0.5135131154420356), (0.2824, 0.686082315686252), (0.28, 0.6962932976599326), (0.0, 0.5135131154420356), (0.28, 0.6962932976599326), (0.0, 0.5135131154420356), (0.286, 0.6706742637480929)]
fuzzy :  None
pembilang :  0.7754867320713145
penyebut :  4.289882521080317
rata-rata tertimbang :  0.18077108831316535
 
[0.0, 0.2824, 0.18077108831316535, 0.28, 0.28, 0.0, 0.288]
mean :  0.18731015547330934
standar de

[0.0, 0.3104, 0.312, 0.20151443792001583, 0.0, 0.308, 0.3156]
mean :  0.20678777684571653
standar deviasi :  0.1359828225079828
fuzzy :  [(0.0, 0.3147017836369086), (0.3104, 0.7480732123492672), (0.312, 0.7413458911783936), (0.20151443792001583, 0.9992484376960512), (0.0, 0.3147017836369086), (0.308, 0.7580820409815122), (0.3156, 0.7260622101759815)]
fuzzy :  None
pembilang :  1.1274993325794906
penyebut :  4.602215359655023
rata-rata tertimbang :  0.24499056312393142
 
[0.0, 0.290068, 0.292092, 0.290484, 0.2912, 0.2896, 0.292836]
mean :  0.24946857142857143
standar deviasi :  0.10185050762599797
fuzzy :  [(0.0, 0.04981837061275222), (0.290068, 0.9236335742957195), (0.292092, 0.9161658238202549), (0.290484, 0.92212347475571), (0.2912, 0.9194942222557426), (0.2896, 0.9253169426790263), (0.292836, 0.9133453082152565)]
fuzzy :  None
pembilang :  1.6065722576782955
penyebut :  5.569897716634462
rata-rata tertimbang :  0.28843837704241465
 
[0.0, 0.2996, 0.3092, 0.3084, 0.25928517975027726,

In [15]:
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk
0,3.890000,4.180000,0.600000,2.508000,2.50,0.343636,0.500,6.68,4.955295,0.200000,...,0.2,0.4,0.0,0.200000,0.0,2.400000,8.574000,0.40000,0.500000,1.71480
1,3.890000,0.487873,0.226988,0.062379,4.83,0.294200,0.966,4.83,5.000000,0.200000,...,0.2,0.4,0.0,0.200000,0.0,2.000000,2.554000,0.40000,0.500000,0.51080
2,3.890000,0.510000,0.200000,0.102000,0.23,0.200000,0.046,0.74,5.000000,0.200000,...,0.2,0.4,0.0,0.200000,0.0,2.000000,1.548000,0.40000,0.500000,0.30960
3,3.890000,4.248542,0.200000,1.417506,10.80,0.600000,6.480,10.80,6.000000,0.600000,...,0.2,0.4,0.0,0.200000,0.0,4.400000,17.530000,0.40000,0.500000,3.50600
4,3.890000,0.228283,0.200000,0.049963,0.08,0.200000,0.016,0.08,5.000000,0.200000,...,0.2,0.4,0.0,0.198212,0.0,1.982118,1.416000,0.35088,0.495529,0.28320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,26.375809,0.000000,0.200000,0.000000,0.40,0.200000,0.080,0.89,5.000000,0.200000,...,0.2,0.4,0.0,0.200000,0.0,2.000000,1.578000,0.40000,0.500000,0.31560
772,55.570000,0.470000,0.200000,0.094000,0.37,0.198212,0.074,0.00,5.000000,0.200000,...,0.2,0.4,0.0,0.200000,0.0,2.000000,1.568000,0.40000,0.500000,0.30122
773,55.570000,0.240000,0.200000,0.000000,0.00,0.200000,0.008,0.28,5.000000,0.200000,...,0.2,0.4,0.0,0.200000,0.0,2.000000,1.456000,0.40000,0.500000,0.29120
774,55.570000,0.200000,0.197913,0.040000,0.00,0.200000,0.000,0.20,5.000000,0.198212,...,0.2,0.4,0.0,0.200000,0.0,2.000000,1.434092,0.40000,0.500000,0.28800


In [16]:
dummy.describe()

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk
count,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,...,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000
mean,19.972087,2.490115,0.349710,1.262446,8.539259,0.309927,6.244432,12.968283,5.044219,0.221771,...,0.204619,0.506860,0.096649,0.215108,0.045361,2.689672,15.585699,0.559815,0.497655,6.497774
std,23.781037,5.628347,0.174105,3.430716,21.591748,0.168341,30.060510,51.062841,0.275010,0.077771,...,0.033818,0.273074,0.522712,0.065478,0.287016,0.855886,39.129726,0.422943,0.015163,37.086441
min,1.185685,0.000000,0.071818,0.000000,0.000000,0.128182,0.000000,0.000000,4.107331,0.170301,...,0.071818,0.256364,0.000000,0.128182,0.000000,1.751218,0.909705,0.256364,0.320455,0.180771
25%,2.370000,0.350055,0.200000,0.000000,0.000000,0.200000,0.000000,0.560000,5.000000,0.200000,...,0.200000,0.400000,0.000000,0.200000,0.000000,2.000000,1.578000,0.400000,0.500000,0.315100
50%,3.890000,0.930000,0.200000,0.147000,0.230000,0.200000,0.051732,1.340000,5.000000,0.200000,...,0.200000,0.400000,0.000000,0.200000,0.000000,2.200000,2.200000,0.400000,0.500000,0.552200
75%,55.570000,2.547500,0.600000,1.321500,2.477500,0.400000,1.327500,7.615000,5.000000,0.200000,...,0.200000,0.400000,0.000000,0.200000,0.000000,3.200000,10.591000,0.400000,0.500000,3.295700
max,59.850000,85.000000,0.600000,51.000000,144.390000,0.600000,758.778000,1268.910000,9.000000,0.600000,...,0.600000,2.400000,9.000000,0.600000,5.400000,5.200000,583.688000,5.800000,0.500000,961.514400


# Klasifikasi

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [18]:
x = dummy.values
y = df.iloc[:, -1].values

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

In [20]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [21]:
classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[120   4]
 [  3  67]]
0.9639175257731959


In [22]:
classifier = MLPClassifier(hidden_layer_sizes=3, learning_rate_init=0.1, max_iter=100)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[124   0]
 [  3  67]]
0.9845360824742269


C:\Users\User\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
